In [3]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import * 
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
TRAIN_PATH = "CovidDataset/Train"
VAL_PATH = "CovidDataset/Test"

In [5]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='softmax'))

model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 128)     36992     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 128)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 110, 110, 128)     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 108, 108, 64)      73792     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 54, 54, 64)       

In [13]:
train_data = image.ImageDataGenerator(rescale = 1./255, horizontal_flip = True,)
test_data = image.ImageDataGenerator(rescale=1./255)

In [10]:
train_generator = train_data.flow_from_directory('CovidDataset/Train',target_size = (224,224), batch_size = 32, class_mode = 'binary')
train_generator.class_indices

Found 224 images belonging to 2 classes.


{'Covid': 0, 'Normal': 1}

In [14]:
validation_generator = test_data.flow_from_directory('CovidDataset/Val', target_size = (224,224), batch_size = 32, class_mode = 'binary')


Found 60 images belonging to 2 classes.


In [15]:
hist = model.fit_generator?

In [23]:
hist = model.fit_generator(train_generator, steps_per_epoch = 8, epochs =10, validation_data=validation_generator) 
                          

Epoch 1/10
8/8 [==============================] - 84s 10s/step - loss: 7.8463 - accuracy: 0.4883 - val_loss: 5.4762 - val_accuracy: 0.5000
Epoch 2/10
8/8 [==============================] - 85s 11s/step - loss: 7.5468 - accuracy: 0.5078 - val_loss: 8.2142 - val_accuracy: 0.5000
Epoch 3/10
8/8 [==============================] - 93s 12s/step - loss: 7.7864 - accuracy: 0.4922 - val_loss: 6.5714 - val_accuracy: 0.5000
Epoch 4/10
8/8 [==============================] - 93s 12s/step - loss: 7.4869 - accuracy: 0.5117 - val_loss: 9.8571 - val_accuracy: 0.5000
Epoch 5/10
8/8 [==============================] - 101s 13s/step - loss: 7.5468 - accuracy: 0.5078 - val_loss: 5.4762 - val_accuracy: 0.5000
Epoch 6/10
8/8 [==============================] - 101s 13s/step - loss: 7.9661 - accuracy: 0.4805 - val_loss: 8.2142 - val_accuracy: 0.5000
Epoch 7/10
8/8 [==============================] - 112s 14s/step - loss: 7.4869 - accuracy: 0.5117 - val_loss: 7.1190 - val_accuracy: 0.5000
Epoch 8/10
8/8 [========

In [25]:
summary=hist.history
print(summary)

{'val_loss': [5.476156711578369, 8.214235305786133, 6.571388244628906, 9.857081413269043, 5.476156711578369, 8.214235305786133, 7.119003772735596, 6.023772716522217, 8.214235305786133, 6.023772716522217], 'val_accuracy': [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], 'loss': [7.846305787563324, 7.546828627586365, 7.786410331726074, 7.486932873725891, 7.54682856798172, 7.966096818447113, 7.486933052539825, 7.906201183795929, 7.367141962051392, 7.8463059067726135], 'accuracy': [0.48828125, 0.5078125, 0.4921875, 0.51171875, 0.5078125, 0.48046875, 0.51171875, 0.484375, 0.51953125, 0.48828125]}


In [26]:
import matplotlib.pyplot as plt
plt.plot(summary["val_accuracy"],label="Validation Accuracy")
plt.plot(summ["accuracy"],label="Training Accuracy")
plt.legend()
plt.xlabel("epochs")
plt.ylabel("Accuracy")
plt.show()

TypeError: 'History' object is not subscriptable